## 사전 훈련된 임베딩으로 테스트

In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data, datasets
from torchtext.vocab import GloVe
import random

In [2]:
SEED = 5
random.seed(SEED)
torch.manual_seed(SEED)

In [3]:
lr = 0.001
EPOCHS = 10
BATCH_SIZE = 64


In [4]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print("cpu와 cuda 중 다음 기기로 학습함:", DEVICE)

cpu와 cuda 중 다음 기기로 학습함: cuda


## Data Prerpocessing
- 토치텍스트가 제공하는 glove.6B.300d를 사용 

텍스트와 label 객체 생성

In [5]:
TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.Field(sequential=False, batch_first=True)

C:\Users\HunbeomBak\.conda\envs\nlp\lib\site-packages\torchtext\data\field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


데이터셋 다운로드하고  훈련 데이터를 8:2로 분리하여 검증 데이터를 생성

In [6]:
trainset, testset = datasets.IMDB.splits(TEXT, LABEL)
#trainset, valset = trainset.split(split_ratio=0.8)


C:\Users\HunbeomBak\.conda\envs\nlp\lib\site-packages\torchtext\data\example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


사전 훈련된 임베딩 베겉를 다운로드 받아 벡터 초기화

In [7]:
max_size = -1
min_freq = 5
emd_dim = 300

TEXT.build_vocab(trainset, vectors=GloVe(name='6B', dim=300), max_size=max_size, min_freq=min_freq) # 단어 집합 생성
LABEL.build_vocab(trainset)

vocab_size = len(TEXT.vocab)
n_classes = 2

print('임베딩 벡터의 개수와 차원 : {} '.format(TEXT.vocab.vectors.shape))
print('단어 집합의 크기 : {}'.format(vocab_size))
print('클래스의 개수 : {}'.format(n_classes))

임베딩 벡터의 개수와 차원 : torch.Size([46159, 300]) 
단어 집합의 크기 : 46159
클래스의 개수 : 2


In [8]:
TEXT.vocab.vectors.shape

torch.Size([46159, 300])

## Dataloader

In [9]:
trainset, valset = trainset.split(split_ratio=0.8)

print('훈련 데이터의 크기 : {}' .format(len(trainset)))
print('검증 데이터의 크기 : {}' .format(len(valset)))
print('테스트 데이터의 크기 : {}' .format(len(testset)))

train_iter, val_iter, test_iter = data.BucketIterator.splits(
        (trainset, valset, testset), batch_size=BATCH_SIZE,
        shuffle=True, repeat=False)

print('훈련 데이터의 미니 배치의 개수 : {}'.format(len(train_iter)))
print('테스트 데이터의 미니 배치의 개수 : {}'.format(len(test_iter)))
print('검증 데이터의 미니 배치의 개수 : {}'.format(len(val_iter)))

훈련 데이터의 크기 : 20000
검증 데이터의 크기 : 5000
테스트 데이터의 크기 : 25000
훈련 데이터의 미니 배치의 개수 : 313
테스트 데이터의 미니 배치의 개수 : 391
검증 데이터의 미니 배치의 개수 : 79


C:\Users\HunbeomBak\.conda\envs\nlp\lib\site-packages\torchtext\data\iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


## Model

In [10]:
class GRU(nn.Module):
    def __init__(self, n_layers, hidden_dim, trained_vector, n_classes, dropout_p=0.2):
        super(GRU, self).__init__()
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        embed_dim = trained_vector.shape[1]
        
        self.embed = nn.Embedding.from_pretrained(trained_vector, freeze=True)
        self.dropout = nn.Dropout(dropout_p)
        self.gru = nn.GRU(embed_dim, self.hidden_dim,
                          num_layers=self.n_layers,
                          batch_first=True)
        self.out = nn.Linear(self.hidden_dim, n_classes)

    def forward(self, x):
        x = self.embed(x)
        h_0 = self._init_state(batch_size=x.size(0)) # 첫번째 히든 스테이트를 0벡터로 초기화
        x, _ = self.gru(x, h_0)  # GRU의 리턴값은 (배치 크기, 시퀀스 길이, 은닉 상태의 크기)
        h_t = x[:,-1,:] # (배치 크기, 은닉 상태의 크기)의 텐서로 크기가 변경됨. 즉, 마지막 time-step의 은닉 상태만 가져온다.
        self.dropout(h_t)
        logit = self.out(h_t)  # (배치 크기, 은닉 상태의 크기) -> (배치 크기, 출력층의 크기)
        return logit

    def _init_state(self, batch_size=1):
        weight = next(self.parameters()).data
        return weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()

In [19]:
model = GRU(2, 256, TEXT.vocab.vectors, 2, dropout_p=0.5).to(DEVICE)

### optimizer

In [20]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)


## Train

### Train code


In [21]:
def train(model, optimizer, train_iter):
    model.train()
    for b, batch in enumerate(train_iter):
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1)  # 레이블 값을 0과 1로 변환
        optimizer.zero_grad()

        logit = model(x)
        loss = F.cross_entropy(logit, y)
        loss.backward()
        optimizer.step()

### evaluate function

In [22]:
def evaluate(model, val_iter):
    """evaluate model"""
    model.eval()
    corrects, total_loss = 0, 0
    for batch in val_iter:
        x, y = batch.text.to(DEVICE), batch.label.to(DEVICE)
        y.data.sub_(1) # 레이블 값을 0과 1로 변환
        logit = model(x)
        loss = F.cross_entropy(logit, y, reduction='sum')
        total_loss += loss.item()
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
    size = len(val_iter.dataset)
    avg_loss = total_loss / size
    avg_accuracy = 100.0 * corrects / size
    return avg_loss, avg_accuracy

### Run train

In [23]:
best_val_loss = None
for e in range(1, EPOCHS+1):
    train(model, optimizer, train_iter)
    val_loss, val_accuracy = evaluate(model, val_iter)

    print("[Epoch: %d] val loss : %5.2f | val accuracy : %5.2f" % (e, val_loss, val_accuracy))

    # 검증 오차가 가장 적은 최적의 모델을 저장
    if not best_val_loss or val_loss < best_val_loss:
        if not os.path.isdir("snapshot"):
            os.makedirs("snapshot")
        torch.save(model.state_dict(), './snapshot/txtclassification_test1.pt')
        best_val_loss = val_loss

[Epoch: 1] val loss :  0.70 | val accuracy : 50.82
[Epoch: 2] val loss :  0.69 | val accuracy : 52.16
[Epoch: 3] val loss :  0.69 | val accuracy : 51.40
[Epoch: 4] val loss :  0.69 | val accuracy : 51.66
[Epoch: 5] val loss :  0.73 | val accuracy : 49.62
[Epoch: 6] val loss :  0.66 | val accuracy : 59.98
[Epoch: 7] val loss :  0.70 | val accuracy : 58.56
[Epoch: 8] val loss :  0.66 | val accuracy : 60.42
[Epoch: 9] val loss :  0.69 | val accuracy : 65.70
[Epoch: 10] val loss :  0.69 | val accuracy : 60.92


In [24]:
test_loss, test_acc = evaluate(model, test_iter)
print('테스트 오차: %5.2f | 테스트 정확도: %5.2f' % (test_loss, test_acc))

테스트 오차:  0.65 | 테스트 정확도: 64.55
